In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time   

In [2]:
base_url = "https://www.pro-football-reference.com/players/"
players = ["Trevor Lawrence", "Zach Wilson", "Trey Lance", "Justin Fields", "Mac Jones", "Kyle Trask", "Kellen Mond", "Davis Mills", "Ian Brook", "Sam Ehlinger"]
player_urls = ["L/LawrTr00/", "W/WilsZa00/", "L/LancTr00/", "F/FielJu00/", "J/JoneMa05/", "T/TrasKy00/", "M/MondKe00/", "M/MillDa02/", "B/BookIa00/", "E/EhliSa00/"]
draft_pick_no = [1, 2, 3, 11, 15, 64, 66, 67, 133, 218]
years = ["2021", "2022", "2023"]

In [3]:
headers = {'User-Agent': 'Mozilla/6.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

obj = {}
obj["Year Drafted"] = []
obj["Name"] = []
obj["Draft Pick"] = []
obj["Date"] = []    
obj["Game Result (W/L)"] = []
obj["Pass Completions"] = []
obj["Passing Attempts"] = []
obj["Passing Yards"] = []
obj["Passing Touchdowns"] = []
obj["Interceptions"] = []
obj["QBR"] = []
obj["Times Sacked"] = []
obj["Rushing Attempts"] = []
obj["Rushing Yards"] = []
obj["Rushing Touchdowns"] = []
obj["Fumbles"] = []

for player, pick, player_name in zip(player_urls, draft_pick_no, players):
    for year in years:
        url = f"{base_url}{player}/gamelog/{year}"
        api_response = requests.get(url, headers=headers)
        time.sleep(3)
        soup = BeautifulSoup(api_response.text, 'html.parser')
        table = soup.find("tbody")

        if table:
            for row in table.find_all("tr"):
                if not row.find("td", class_="center"):
                    obj["Year Drafted"].append("2021")
                    obj["Name"].append(player_name)
                    obj["Draft Pick"].append(pick)
                    
                    # Game date
                    try:
                        obj["Date"].append(row.find("td", {"data-stat": "game_date"}).text)
                    except (AttributeError, ValueError):
                        obj["Date"].append(np.nan)

                    # Game result
                    try:
                        obj["Game Result (W/L)"].append(row.find("td", {"data-stat": "game_result"}).text[0])
                    except (AttributeError, IndexError):
                        obj["Game Result (W/L)"].append(np.nan)

                    # Passing completions
                    try:
                        #passing_completions = row.find("td", {"data-stat": "pass_cmp"}).text
                        obj["Pass Completions"].append(int(row.find("td", {"data-stat": "pass_cmp"}).text))
                    except (AttributeError, ValueError):
                        obj["Pass Completions"].append(0.0)

                    # Passing attempts
                    try:
                        obj["Passing Attempts"].append(int(row.find("td", {"data-stat": "pass_att"}).text))
                    except (AttributeError, ValueError):
                        obj["Passing Attempts"].append(0.0)

                    # Passing yards
                    try:
                        obj["Passing Yards"].append(int(row.find("td", {"data-stat": "pass_yds"}).text))
                    except (AttributeError, ValueError):
                        obj["Passing Yards"].append(0.0)

                    # Passing touchdowns
                    try:
                        obj["Passing Touchdowns"].append(int(row.find("td", {"data-stat": "pass_td"}).text))
                    except (AttributeError, ValueError):
                        obj["Passing Touchdowns"].append(0.0)

                    # Interceptions
                    try:
                        obj["Interceptions"].append(int(row.find("td", {"data-stat": "pass_int"}).text))
                    except (AttributeError, ValueError):
                        obj["Interceptions"].append(0.0)

                    # Passer rating
                    try:
                        obj["QBR"].append(float(row.find("td", {"data-stat": "pass_rating"}).text))
                    except (AttributeError, ValueError):
                        obj["QBR"].append(0.0)

                    # Times sacked
                    try:
                        obj["Times Sacked"].append(int(row.find("td", {"data-stat": "pass_sacked"}).text))
                    except (AttributeError, ValueError):
                        obj["Times Sacked"].append(0.0)

                    # Rushing attempts
                    try:
                        obj["Rushing Attempts"].append(int(row.find("td", {"data-stat": "rush_att"}).text))
                    except (AttributeError, ValueError):
                        obj["Rushing Attempts"].append(0.0)

                    # Rushing yards
                    try:
                        obj["Rushing Yards"].append(int(row.find("td", {"data-stat": "rush_yds"}).text))
                    except (AttributeError, ValueError):
                        obj["Rushing Yards"].append(0.0)

                    # Rushing touchdowns
                    try:
                        obj["Rushing Touchdowns"].append(int(row.find("td", {"data-stat": "rush_td"}).text))
                    except (AttributeError, ValueError):
                        obj["Rushing Touchdowns"].append(0.0)

                    # Fumbles
                    try:
                        obj["Fumbles"].append(int(row.find("td", {"data-stat": "fumbles"}).text))
                    except (AttributeError, ValueError):
                        obj["Fumbles"].append(0.0)

df = pd.DataFrame(obj)
df.to_csv('2021.csv')